In [14]:
import pandas as pd
import regex as re
import string
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from transformers import pipeline, RobertaTokenizer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/michelleuni/nltk_data...


True

In [11]:
df = pd.read_csv('data/all_data.csv', index_col = 0)
#df.index.name = None

In [12]:
df.head(5)

,category,title,tags,age_rating,rating_description,description,representation,duration_txt,duration_sec,first_broadcast,synopsis_small,synopsis_medium,synopsis_large,image,showId,description_all,vader_scores,vader_polarity,vader_sentiment
0,CBBC,Operation Ouch! - Series 9: 11. Blink and You’...,"BBC, iPlayer, TV, Operation Ouch!, Series 9: 1...",PG,"injury detail, rude humour, threat",Why is Dr Xand dressed as a bird in the lab? F...,3.0,28 mins,1692,5pm 17 Mar 2021,Why is Dr Xand dressed as a bird in the lab? F...,Why is Dr Xand dressed as a bird in the lab? F...,Why is Dr Xand dressed as a bird in the lab? F...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,1,Operation Ouch! - Series 9: 11. Blink and You’...,"{'neg': 0.133, 'neu': 0.82, 'pos': 0.046, 'com...",-0.7044,Negative
1,CBBC,The Worst Witch - Series 4: 1. The Three Impos...,"BBC, iPlayer, TV, The Worst Witch, Series 4: 1...",PG,threat,A botched potions experiment leaves Mildred wi...,3.0,28 mins,1693,27 Jan 2020,A botched potions experiment leaves Mildred wi...,When a botched potions experiment leaves Mildr...,No data found,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,2,The Worst Witch - Series 4: 1. The Three Impos...,"{'neg': 0.199, 'neu': 0.775, 'pos': 0.026, 'co...",-0.8658,Very negative
2,CBBC,Shaun the Sheep - Series 4: 6. The Smelly Farmer,"BBC, iPlayer, TV, Shaun the Sheep, Series 4: 6...",U,clean,"Shaun, Bitzer and the flock decide to give the...",NaN,7 mins,425,3:50pm 10 Feb 2014,"Shaun, Bitzer and the flock decide to give the...",Animated series. The farmer's personal hygiene...,The farmer's personal hygiene leaves much to b...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,3,Shaun the Sheep - Series 4: 6. The Smelly Farm...,"{'neg': 0.0, 'neu': 0.946, 'pos': 0.054, 'comp...",0.2732,Positive
3,CBBC,Pokémon: Black and White - Movies: Black - Vic...,"BBC, iPlayer, TV, Pokémon: Black and White, Mo...",PG,contains mild violence and threat,Can Ash awaken Reshiram to help him rescue Vic...,NaN,92 mins,5497,3 Dec 2021,Can Ash awaken Reshiram to help him rescue Vic...,Can Ash awaken legendary Pokémon Reshiram to h...,"When Ash, Iris and Cilan enter a battle compet...",https://ichef.bbci.co.uk/images/ic/1200x675/p0...,4,Pokémon: Black and White - Movies: Black - Vic...,"{'neg': 0.147, 'neu': 0.548, 'pos': 0.305, 'co...",0.7941,Very positive
4,CBBC,"Deadly 60 - Series 2: 1. Baja California, Mexico","BBC, iPlayer, TV, Deadly 60, Series 2: 1. Baja...",PG,upsetting scenes,"In Mexico, Steve Backshall takes to the water ...",1.0,28 mins,1680,5:15pm 8 Jul 2010,"In Mexico, Steve Backshall takes to the water ...","Wildlife series. On a trip to Mexico, Steve Ba...",Steve travels to the deadliest desert on the p...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,5,"Deadly 60 - Series 2: 1. Baja California, Mexi...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,Neutral


# VADER analysis
To implement the positivity value we use Sentiment analysis with VADER to calculate the polarity score and sentiment. They are added to the df and then safed again in the all_data.csv

In [15]:
# Import necessary libraries
from nltk.sentiment.vader import SentimentIntensityAnalyzer

vader_model = SentimentIntensityAnalyzer()

# Combine the columns we want VADER to use for the polarity calculation
df['description_all'] =  df['title'].astype(str) + df['synopsis_medium'].astype(str) + df['description'].astype(str) + df['rating_description'].astype(str)

# Applying VADER model to analyze sentiment
df["vader_scores"] = df["description_all"].apply(lambda text: vader_model.polarity_scores(text))

# Assigning sentiment labels based on VADER scores
df["vader_polarity"] = df["vader_scores"].apply(lambda score_dict: score_dict["compound"])
df["vader_sentiment"] = pd.cut(df["vader_polarity"],
                                     bins=[-1.0, -0.75, -0.25, 0.25, 0.75, 1.0],
                                     labels=["Very negative", "Negative", "Neutral", "Positive", "Very positive"])

# Display df to check the changes
df.head(2)

,category,title,tags,age_rating,rating_description,description,representation,duration_txt,duration_sec,first_broadcast,synopsis_small,synopsis_medium,synopsis_large,image,showId,description_all,vader_scores,vader_polarity,vader_sentiment
0,CBBC,Operation Ouch! - Series 9: 11. Blink and You’...,"BBC, iPlayer, TV, Operation Ouch!, Series 9: 1...",PG,"injury detail, rude humour, threat",Why is Dr Xand dressed as a bird in the lab? F...,3.0,28 mins,1692,5pm 17 Mar 2021,Why is Dr Xand dressed as a bird in the lab? F...,Why is Dr Xand dressed as a bird in the lab? F...,Why is Dr Xand dressed as a bird in the lab? F...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,1,Operation Ouch! - Series 9: 11. Blink and You’...,"{'neg': 0.133, 'neu': 0.82, 'pos': 0.046, 'com...",-0.7044,Negative
1,CBBC,The Worst Witch - Series 4: 1. The Three Impos...,"BBC, iPlayer, TV, The Worst Witch, Series 4: 1...",PG,threat,A botched potions experiment leaves Mildred wi...,3.0,28 mins,1693,27 Jan 2020,A botched potions experiment leaves Mildred wi...,When a botched potions experiment leaves Mildr...,No data found,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,2,The Worst Witch - Series 4: 1. The Three Impos...,"{'neg': 0.199, 'neu': 0.775, 'pos': 0.026, 'co...",-0.8658,Very negative


# Clean Text

In [5]:
#Function to clean-up texts
def clean_text(x):

    # remove punctuation
    x = x.translate(str.maketrans(' ', ' ', string.punctuation))

    # lowercase
    x = x.lower()

    # remove — and ’
    x = re.sub("—", " ", x)
    x = re.sub("’", " ", x)
    x = re.sub("–", " ", x)
    x = re.sub("…", " ", x)
    x = re.sub("“", " ", x)
    x = re.sub("”", " ", x)
    #x = re.sub("\\x", " ", x)

    # strip excessive whitespaces
    x = x.strip()

    # remove stopwords
    # x = [token for token in x if not token in stop_words]


    return x

In [6]:
description_cleaned = df['description'].apply(clean_text)
synopsis_small_cleaned = df['synopsis_small'].apply(clean_text)
print(description_cleaned.equals(synopsis_small_cleaned))

True


Since the two columns are identical, one will be dropped.

# Removing columns

In [22]:
df['synopsis'] = np.where(df['synopsis_large'] != 'No data found', df['synopsis_large'],
                                   np.where(df['synopsis_medium'] != 'No data found', df['synopsis_medium'],
                                            df['synopsis_small']))

In [23]:
df.drop(columns=['synopsis_large', 'synopsis_small'], inplace=True)

In [24]:
df

,category,title,tags,age_rating,rating_description,description,representation,duration_txt,duration_sec,first_broadcast,synopsis_medium,image,showId,description_all,vader_scores,vader_polarity,vader_sentiment,sentiment,rating_description_score,synopsis
0,CBBC,Operation Ouch! - Series 9: 11. Blink and You’...,"BBC, iPlayer, TV, Operation Ouch!, Series 9: 1...",PG,"injury detail, rude humour, threat",Why is Dr Xand dressed as a bird in the lab? F...,3.0,28 mins,1692,5pm 17 Mar 2021,Why is Dr Xand dressed as a bird in the lab? F...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,1,Operation Ouch! - Series 9: 11. Blink and You’...,"{'neg': 0.133, 'neu': 0.82, 'pos': 0.046, 'com...",-0.7044,Negative,0.9892,-0.9966,Why is Dr Xand dressed as a bird in the lab? F...
1,CBBC,The Worst Witch - Series 4: 1. The Three Impos...,"BBC, iPlayer, TV, The Worst Witch, Series 4: 1...",PG,threat,A botched potions experiment leaves Mildred wi...,3.0,28 mins,1693,27 Jan 2020,When a botched potions experiment leaves Mildr...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,2,The Worst Witch - Series 4: 1. The Three Impos...,"{'neg': 0.199, 'neu': 0.775, 'pos': 0.026, 'co...",-0.8658,Very negative,0.9973,-0.9921,When a botched potions experiment leaves Mildr...
2,CBBC,Shaun the Sheep - Series 4: 6. The Smelly Farmer,"BBC, iPlayer, TV, Shaun the Sheep, Series 4: 6...",U,clean,"Shaun, Bitzer and the flock decide to give the...",NaN,7 mins,425,3:50pm 10 Feb 2014,Animated series. The farmer's personal hygiene...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,3,Shaun the Sheep - Series 4: 6. The Smelly Farm...,"{'neg': 0.0, 'neu': 0.946, 'pos': 0.054, 'comp...",0.2732,Positive,-0.9901,0.9973,The farmer's personal hygiene leaves much to b...
3,CBBC,Pokémon: Black and White - Movies: Black - Vic...,"BBC, iPlayer, TV, Pokémon: Black and White, Mo...",PG,contains mild violence and threat,Can Ash awaken Reshiram to help him rescue Vic...,NaN,92 mins,5497,3 Dec 2021,Can Ash awaken legendary Pokémon Reshiram to h...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,4,Pokémon: Black and White - Movies: Black - Vic...,"{'neg': 0.147, 'neu': 0.548, 'pos': 0.305, 'co...",0.7941,Very positive,0.9946,-0.9782,"When Ash, Iris and Cilan enter a battle compet..."
4,CBBC,"Deadly 60 - Series 2: 1. Baja California, Mexico","BBC, iPlayer, TV, Deadly 60, Series 2: 1. Baja...",PG,upsetting scenes,"In Mexico, Steve Backshall takes to the water ...",1.0,28 mins,1680,5:15pm 8 Jul 2010,"Wildlife series. On a trip to Mexico, Steve Ba...",https://ichef.bbci.co.uk/images/ic/1200x675/p0...,5,"Deadly 60 - Series 2: 1. Baja California, Mexi...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,Neutral,0.9985,-0.9990,Steve travels to the deadliest desert on the p...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,Science & Nature,Born to Be Wild - Series 2: Episode 1,"BBC, iPlayer, TV, Born to Be Wild, Series 2: E...",12,dangerous behaviour,A look behind the scenes at the Scottish SPCA’...,1.0,58 mins,3480,8pm 19 Jul 2021,April helps hand-rear Padme the orphaned fox c...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,236,Born to Be Wild - Series 2: Episode 1April hel...,"{'neg': 0.0, 'neu': 0.821, 'pos': 0.179, 'comp...",0.8316,Very positive,0.9966,-0.9987,It’s the start of spring at the SSPCA Wildlife...
236,Science & Nature,Shark - Episode 3,"BBC, iPlayer, TV, Shark, Episode 3",15,scenes of animal threat,An international team of scientists is trying ...,NaN,58 mins,3480,9pm 21 May 2015,Wildlife series on the sharks of the world. Sh...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,237,Shark - Episode 3Wildlife series on the sharks...,"{'neg': 0.136, 'neu': 0.864, 'pos': 0.0, 'comp...",-0.7541,Very negative,0.9978,-0.9918,Sharks are among the most misunderstood predat...
237,Science & Nature,Greta Thunberg: A Year to Change the World - S...,"BBC, iPlayer, TV, Greta Thunberg: A Year to Ch...",U,clean,Greta sets off to the annual 

# Child Appropriateness Score

This section prepares the dataset for the metrics that are necessary for the Child Appropriateness Score

## Sentiment Analysis

The sentiment_analysis method returns the label and the confidence/accuracy of the classification.
    If the clasification is positive, the method will simply return the accuracy score. If it is negative, it will 
    return the negative of the score.

In [16]:
sentiment_analysis = pipeline("sentiment-analysis", model="siebert/sentiment-roberta-large-english")

In [17]:
def analyse_sentiment(text):
    output = sentiment_analysis(text)
    score = round(output[0]['score'], 4) if output else None #round the sentiment score to 4 decimal places

    if output[0]['label'] == 'NEGATIVE':
        return -score
    return score

In [18]:
df['sentiment'] = df['synopsis_medium'].apply(analyse_sentiment)

In [19]:
df['rating_description'].fillna('No description available', inplace=True)

In [20]:
df['rating_description_score'] = df['rating_description'].apply(analyse_sentiment)

## Term Frequency

Specific content within the synopsis that could be potentially inappropriate for children, such as violence, sexual content, drug references, profanity, etc. is searched for and assessed through a term frequency matrix, indicating the portion of the unsuitable terms present within the description. The same terms will be used for all age groups but the final weighting will be different.

In [25]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def remove_stopwords_and_lemmatize(x):
    tokens = word_tokenize(x)
    
    tokens = [lemmatizer.lemmatize(token.lower()) for token in tokens if token.lower() not in stop_words]
    
    result = []
    for i, token in enumerate(tokens):
        if i > 0 and all(char in string.punctuation for char in token):
            result[-1] += token
        else:
            result.append(token)
    
    return ' '.join(result)

df['synopsis_nostopwords'] = df['synopsis'].apply(remove_stopwords_and_lemmatize)

In [26]:
violence_terms = ['blood', 'gore', 'kill', 'murder', 'torture', 'assault, weapon', 'war']
sexual_terms = ['sex', 'pornography', 'nude', 'erotic']
drug_terms = ['cocaine', 'heroin', 'weed', 'alcohol', 'drug']
scary_terms = ['horror', 'terror', 'panic', 'frighten', 'violence']
mature_terms = ['suicide', 'depression', 'mental illness', 'abuse', 'death']
language_terms = ['language', 'swear', 'rude']
unsuitable_terms = violence_terms + sexual_terms + drug_terms + scary_terms + mature_terms + language_terms

In [27]:
"""
    This method counts each unsuitable word found in a sentence and returns the negated fraction of the unsuitable words over
    the total number of words in the sentence.
"""
def vectorize_row(text, unsuitable_terms):
    if isinstance(text, float):
        return 0  
    
    words = str(text).split()  
    unsuitable_terms_count = 0
    
    for word in words:
        if word.lower() in unsuitable_terms:
            unsuitable_terms_count += 1
    
    total_terms = len(words)
    
    return -(unsuitable_terms_count / total_terms)


In [28]:
df['unsuitable_ratio'] = df.apply(lambda row: vectorize_row(row['synopsis_nostopwords'], unsuitable_terms), axis=1)
df['unsuitable_ratio2'] = df.apply(lambda row: vectorize_row(row['rating_description'], unsuitable_terms), axis=1)

In [29]:
df.head()

,category,title,tags,age_rating,rating_description,description,representation,duration_txt,duration_sec,first_broadcast,...,description_all,vader_scores,vader_polarity,vader_sentiment,sentiment,rating_description_score,synopsis,synopsis_nostopwords,unsuitable_ratio,unsuitable_ratio2
0,CBBC,Operation Ouch! - Series 9: 11. Blink and You’...,"BBC, iPlayer, TV, Operation Ouch!, Series 9: 1...",PG,"injury detail, rude humour, threat",Why is Dr Xand dressed as a bird in the lab? F...,3.0,28 mins,1692,5pm 17 Mar 2021,...,Operation Ouch! - Series 9: 11. Blink and You’...,"{'neg': 0.133, 'neu': 0.82, 'pos': 0.046, 'com...",-0.7044,Negative,0.9892,-0.9966,Why is Dr Xand dressed as a bird in the lab? F...,dr xand dressed bird lab? find blink today ’ e...,-0.0,-0.2
1,CBBC,The Worst Witch - Series 4: 1. The Three Impos...,"BBC, iPlayer, TV, The Worst Witch, Series 4: 1...",PG,threat,A botched potions experiment leaves Mildred wi...,3.0,28 mins,1693,27 Jan 2020,...,The Worst Witch - Series 4: 1. The Three Impos...,"{'neg': 0.199, 'neu': 0.775, 'pos': 0.026, 'co...",-0.8658,Very negative,0.9973,-0.9921,When a botched potions experiment leaves Mildr...,botched potion experiment leaf mildred whole n...,-0.0,-0.0
2,CBBC,Shaun the Sheep - Series 4: 6. The Smelly Farmer,"BBC, iPlayer, TV, Shaun the Sheep, Series 4: 6...",U,clean,"Shaun, Bitzer and the flock decide to give the...",NaN,7 mins,425,3:50pm 10 Feb 2014,...,Shaun the Sheep - Series 4: 6. The Smelly Farm...,"{'neg': 0.0, 'neu': 0.946, 'pos': 0.054, 'comp...",0.2732,Positive,-0.9901,0.9973,The farmer's personal hygiene leaves much to b...,farmer 's personal hygiene leaf much desired. ...,-0.0,-0.0
3,CBBC,Pokémon: Black and White - Movies: Black - Vic...,"BBC, iPlayer, TV, Pokémon: Black and White, Mo...",PG,contains mild violence and threat,Can Ash awaken Reshiram to help him rescue Vic...,NaN,92 mins,5497,3 Dec 2021,...,Pokémon: Black and White - Movies: Black - Vic...,"{'neg': 0.147, 'neu': 0.548, 'pos': 0.305, 'co...",0.7941,Very positive,0.9946,-0.9782,"When Ash, Iris and Cilan enter a battle compet...","ash, iris cilan enter battle competition eindo...",-0.0,-0.2
4,CBBC,"Deadly 60 - Series 2: 1. Baja California, Mexico","BBC, iPlayer, TV, Deadly 60, Series 2: 1. Baja...",PG,upsetting scenes,"In Mexico, Steve Backshall takes to the water ...",1.0,28 mins,1680,5:15pm 8 Jul 2010,...,"Deadly 60 - Series 2: 1. Baja California, Mexi...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,Neutral,0.9985,-0.9990,Steve travels to the deadliest desert on the p...,"steve travel deadliest desert planet, baja pen...",-0.0,-0.0


## Age Rating per Movie

Currently, the age rating has no numeric value. Now, a score will be assigned based on the rating. The numerical value represents the proportion of children aged between 4 and 17 that can safely view the content. 

In [30]:
rating_mapping = {'U': 1, 'PG': 0.71, '12': 0.57, '12A': 0.57, '15':0.21, '18': 0.0}
def age_rating_weighting(rating):
    return rating_mapping(rating, 0.0)

df['age_rating_numeric'] = df['age_rating'].map(rating_mapping)

## Child Appropriateness Score

$sentiment$ is the Sentiment Analysis score for the synopsis <br>
$freq1$ is the proportion of the synopsis that contains any of the unsuitable terms <br>
$freq2$ is the proportion of the age description that contains any unsuitable terms <br>
$rating$ is the score for the Age Rating <br>
$rating\_desc\_score$ is the Sentiment Anlaysis score for the rating description <br>

$s$: is the weight for the Sentiment Analysis score <br>
$t1$: is the weight for the Term Frequency score for the synopses <br>
$t2$: is the weight for the Term Frequency score for the age description<br>
$r1$: is the weight for the Age Rating score <br>
$r2$: is the weight for the Age Rating Description score <br>

In [31]:
def child_appropriateness_score(sentiment, freq1, freq2, rating, rating_desc_score, s, t1, t2, r1, r2):
    return s*sentiment + t1*freq1 + t2*freq2 + r1*rating + r2*rating_desc_score

In [32]:
# the weighting for each age group
w48 = [0.2, 0.15, 0.15, 0.3, 0.2]
w911 = [0.20, 0.125, 0.125, 0.30, 0.25]
w1214 = [0.20, 0.10, 0.10, 0.25, 0.35]
w1517 = [0.15, 0.075, 0.075, 0.20, 0.50]

In [33]:
df['cas48'] = df.apply(
    lambda row: child_appropriateness_score(row['sentiment'], row['unsuitable_ratio'], row['unsuitable_ratio2'],row['age_rating_numeric'], 
                                            row['rating_description_score'], w48[0], w48[1], w48[2], w48[3], w48[4]), axis=1)

df['cas911'] = df.apply(
    lambda row: child_appropriateness_score(row['sentiment'], row['unsuitable_ratio'], row['unsuitable_ratio2'],row['age_rating_numeric'],
                                            row['rating_description_score'], w911[0], w911[1], w911[2], w911[3], w911[4]), axis=1)

df['cas1214'] = df.apply(
    lambda row: child_appropriateness_score(row['sentiment'], row['unsuitable_ratio'], row['unsuitable_ratio2'], row['age_rating_numeric'],
                                            row['rating_description_score'], w1214[0], w1214[1], w1214[2], w1214[3], w1214[4]), axis=1)

df['cas1517'] = df.apply(
    lambda row: child_appropriateness_score(row['sentiment'], row['unsuitable_ratio'], row['unsuitable_ratio2'],row['age_rating_numeric'],
                                           row['rating_description_score'], w1517[0], w1517[1], w1517[2], w1517[3], w1517[4]), axis=1)

In [34]:
# recommendations for 4-6
recommendations4_8 = df[['category', 'title', 'tags', 'age_rating', 'image','description', 'cas48', 'vader_sentiment']].sort_values(by='cas48', ascending=False)
filtered4_8 = recommendations4_8[recommendations4_8['cas48'] >= 0.6]
filtered4_8.to_csv('data/recommendations/ages4_8.csv')

# recommendations for 9-11 and under
recommendations9_11 = df[['category', 'title', 'tags', 'age_rating', 'image','description', 'cas911', 'vader_sentiment']].sort_values(by='cas911', ascending=False)
filtered9_11 = recommendations9_11[recommendations9_11['cas911'] >= 0.6]
filtered9_11.to_csv('data/recommendations/ages9_11.csv')

# recommendations for 12-14
recommendations12_14 = df[['category', 'title', 'tags', 'age_rating', 'image','description', 'cas1214', 'vader_sentiment']].sort_values(by='cas1214', ascending=False)
filtered12_14 = recommendations12_14[recommendations12_14['cas1214'] >= 0.6]
filtered12_14.to_csv('data/recommendations/ages12_14.csv')

# recommendations for 15-17
recommendations15_17 = df[['category', 'title', 'tags', 'age_rating', 'image','description', 'cas1517', 'vader_sentiment']].sort_values(by='cas1517', ascending=False)
filtered15_17 = recommendations15_17[recommendations15_17['cas1517'] >= 0.6]
filtered15_17.to_csv('data/recommendations/ages15_17.csv')

# Similarity Testing

### Vector calculation

Vectorization with the use of TF-IDF (Term Frequency-Inverse Document Frequency) we vectorize the combined text features of each movie. Therefore we are using the sklearn library.


In [37]:
# Import necessary libraries
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    strip_accents='unicode',
    lowercase=True,
    stop_words='english',
)

# Use the Description column, earlier we removed the title words from this column
movie_vectors = vectorizer.fit_transform(df['description_all']) #description_all = title + description + synopsis_medium this is specified by the code for VADER


### Similarity calculation
Now we will perform the cosine similarity calculation, this is based on our calculation of the vectorized features above.


In [38]:
# Import necessary libraries
from sklearn.metrics.pairwise import cosine_similarity

# Test the function, the similarity of the same row should be 1 --> see the output it is 1
print(cosine_similarity(movie_vectors[0], movie_vectors[0]))

# Calculate similarities between all movies
similarity_matrix = cosine_similarity(movie_vectors, movie_vectors)

# The shape of the similarities equals: N_movies x N_movies --> this also shows how many movies/content is in our dataset
similarity_matrix.shape

[[1.]]


(240, 240)

### Diversity calculation
Next is calculating the diversity score. For each movie we will do: $\text{Diversity Score =}$ $1 - \text{average cosine similarity}$ with all other movies.

In [39]:
# Calculate the diversity score
similarities_df = pd.DataFrame(similarity_matrix, index=df.index, columns=df.index)
similarity_avg = similarities_df.mean(axis=1)
df['diversity'] = 1-similarity_avg

### Recommendation Generation

  - Below are the functions to calculate the recommendations.

In [28]:
# Import necessary libraries
import functools
from sklearn.preprocessing import MinMaxScaler

# Gets similarity scores for all other movies
def get_similarities(movie_index):
    similarities = similarities_df.loc[movie_index]
    similarities.drop(movie_index, inplace=True)
    return similarities.rename('similarity').to_frame()

# Scales relevant features to domain [0, 1]
def scale_features(movie_df):
    scaler = MinMaxScaler((0,1))
    scaler.fit(movie_df.similarity.to_frame())
    movie_df['similarity_scaled'] = scaler.transform(movie_df.similarity.to_frame())
    scaler.fit(movie_df.diversity.to_frame())
    movie_df['diversity_scaled'] = scaler.transform(movie_df.diversity.to_frame())
    return movie_df

# Calculates weighter average for relevant (scaled) features
def weighted_score(movie, similarity_weight, diversity_weight):
    sw = movie ['similarity_scaled'] * similarity_weight
    dw = movie['diversity_scaled'] * diversity_weight
    total_weights = similarity_weight + diversity_weight
    return (sw + dw) / total_weights

# recommmends the movies + uses function from above
def recommend_movies(movie_index, n_results=10, diversity_factor=0.5, similarity_factor=1):
    # Filter movies with the sentiement thats is given in streamlit
    filtered_df = df[df['vader_sentiment'] == "Positive"] # TODO "Positive" --> should be changed to output streamlit  !!!!!!!
    similarities = get_similarities(movie_index)
    # DataFrame with relevant features for filtered movies
    movie_df = filtered_df.join(similarities)
    movie_df = scale_features(movie_df)
    # Calculate the weighted score
    weight_func = functools.partial(weighted_score, 
                                    similarity_weight=similarity_factor,
                                    diversity_weight=diversity_factor)
    movie_df['recommender_score'] = movie_df.apply(weight_func, axis='columns')
    return movie_df.sort_values('recommender_score', ascending=False).head(n_results)

### Analysis
We used a diversity of 0.5 for the recommendation systems to ensure that the recommendations are not only new episodes but have a bit of a variation in there.

In [29]:
# Insert index column
df.insert(0, 'index', df.index)
df['index'] = df['index'].astype(object)

# Function to display the recommendations
def display_recommendations(movie_index, diversity_factors=[0.5]):
    recommendations_list = []

    for f in diversity_factors:
        recommendations = recommend_movies(movie_index, 10, f)[['index', 'recommender_score', 'vader_polarity', 'vader_sentiment']].values.tolist()
        for rec in recommendations:
            recommendations_list.append({'index': rec[0], 'recommender_score': rec[1], 'vader_polarity': rec[2], 'vader_sentiment': rec[3]})

    # Convert the list of dictionaries into a DataFrame
    df_recommendations = pd.DataFrame(recommendations_list)

    # Convert index to integers
    df_recommendations['index'] = df_recommendations['index'].astype(int)
    df_recommendations['index'] = df_recommendations['index'].astype(str)
    df_recommendations['recommender_score'] = df_recommendations['recommender_score'].astype(object)
    print('Recommendations for movie: "{}"\n'.format(random_movie.title.values[0]))
    print('Genre: "{}"\n'.format(random_movie.category.values[0]))
    print('Sentiment: "{}"\n'.format(random_movie.vader_sentiment.values[0]))
    df_recommendations.to_csv('data/recommendation_data.csv', index=False, sep=';')
    return df_recommendations

# Select a random movie this will be done once and also used for the other recoomendation functions (positive and very positive)
random_movie = df.sample(1) # TODO change this to the profiles Zang made
movie_index = random_movie.index[0]

# Show the recommendations
df_recommendations = display_recommendations(movie_index)
df_recommendations.head(10)

Recommendations for movie: "MasterChef: The Professionals - Series 14: Episode 13 - Signed"

Genre: "Signed"

Polarity: "0.0"



,index,recommender_score,vader_polarity,vader_sentiment
0,19,0.820428,0.6705,Positive
1,16,0.442904,0.7271,Positive
2,117,0.414548,0.5994,Positive
3,145,0.40158,0.6249,Positive
4,203,0.365202,0.6369,Positive
5,64,0.33875,0.5574,Positive
6,207,0.333333,0.3365,Positive
7,163,0.319099,0.5267,Positive
8,77,0.314811,0.5106,Positive
9,56,0.297036,0.4404,Positive
